In [1]:
import requests 
import pandas as pd
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

/Users/crystalbravo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/crystalbravo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = df.iloc[:300]

In [8]:
df.iloc[0]

answer_llm     You can sign up for the course by visiting the...
answer_orig    Machine Learning Zoomcamp FAQ\nThe purpose of ...
document                                                0227b872
question                     Where can I sign up for the course?
course                                 machine-learning-zoomcamp
Name: 0, dtype: object

**Q1. Getting the embeddings model

In [9]:
from sentence_transformers import SentenceTransformer
model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

/Users/crystalbravo/opt/anaconda3/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
v_llm = embedding_model.encode(df.iloc[0].answer_llm)
# v_orig = embedding_model.encode(df.iloc[0].answer_orig)

# v_llm.dot(v_orig)

In [12]:
v_llm[0]

-0.42244658

**Q2. Computing the dot product

In [17]:
from tqdm.auto import tqdm
evaluations = []
for i in tqdm(range(len(df))):
    v_llm = embedding_model.encode(df.iloc[i].answer_llm)
    v_orig = embedding_model.encode(df.iloc[i].answer_orig)
    dot = v_llm.dot(v_orig)
    evaluations.append(dot)


  0%|          | 0/300 [00:00<?, ?it/s]

In [19]:
df["evaluations"] = evaluations
df["evaluations"].describe()

count    300.000000
mean      27.495996
std        6.384743
min        4.547925
25%       24.307841
50%       28.336861
75%       31.674304
max       39.476013
Name: evaluations, dtype: float64

In [48]:
df["evaluations"].quantile(q=0.75)

31.674304008483887

**Q3. Computing the cosine

In [20]:
import numpy as np

In [22]:
def get_v_norm(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [23]:
similarity = []
for i in tqdm(range(len(evaluations))):
    v_llm = embedding_model.encode(df.iloc[i].answer_llm)
    v_llm = get_v_norm(v_llm)
    v_orig = embedding_model.encode(df.iloc[i].answer_orig)
    v_orig = get_v_norm(v_orig)
    dot = v_llm.dot(v_orig)
    similarity.append(dot)

  0%|          | 0/300 [00:00<?, ?it/s]

In [24]:
df["similarity"] = similarity
df["similarity"].describe()

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: similarity, dtype: float64

In [47]:
df["similarity"].quantile(q=0.75)

0.8362347632646561

**Q4. Rouge

In [25]:
%pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


    extract-msg (<=0.29.*)
                 ~~~~~~~^

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: /Users/crystalbravo/opt/anaconda3/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
r = df.iloc[10]
r

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
evaluations                                            32.344711
similarity                                              0.777956
Name: 10, dtype: object

In [40]:
from rouge import Rouge
rouge_scorer = Rouge()
def get_scores(r):
    return rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores = get_scores(r)
scores['rouge-1']['f']

0.45454544954545456

**Q5. Average rouge score

In [41]:
def average_rouge_score(scores):
    avg = (scores['rouge-2']['f'] + scores['rouge-1']['f'] + scores['rouge-l']['f'] ) / 3
    return avg

In [42]:
average_rouge_score(score)

0.35490034990035496

**Q6. Average rouge score for all the data points

In [43]:
rouge_2_scores = []
for i in tqdm(range(len(df))):
    rec = df.iloc[i]
    scores = get_scores(rec)
    rouge_2 = scores['rouge-2']['f']
    rouge_2_scores.append(rouge_2)

  0%|          | 0/300 [00:00<?, ?it/s]

In [45]:
df["rouge_2_scores"] = rouge_2_scores
df["rouge_2_scores"].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge_2_scores, dtype: float64

In [46]:
df["rouge_2_scores"].mean()

0.20696501983423318